In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
style.use('ggplot')

df = pd.read_excel('PL_2019-2020.xlsx')
df

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DIVERTED_AREA,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID
0,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,13.0,NaN,...,0.0,0.0,4.0,0.0,187.763,46.941,426,914297,11,12
1,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,4.0,NaN,...,0.0,0.0,5.0,0.0,99.020,19.804,361,984477,10,9
2,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,5.0,NaN,...,0.0,0.0,5.0,0.0,40.679,8.136,347,985669,10,10
3,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,7.0,NaN,...,6.0,0.0,6.0,0.0,0.000,NaN,524,903118,19,26
4,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100004,VENKAPPA SHASAPPA KAMADINNI,1.0,NaN,...,0.0,0.0,5.0,0.0,57.281,11.456,387,917920,20,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,365911,KARIYAPPA HUCHAPPA HUGGI,2.0,NaN,...,0.0,2.0,0.0,2.0,0.000,NaN,65,979755,238108,327730
53421,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,369510,SAKRAPPA YALLAPPA HAVOJI,1.0,NaN,...,0.0,0.0,1.0,0.0,21.206,21.206,404,1011696,246928,337841
53422,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,1.0,NaN,...,0.0,0.0,1.0,0.0,23.849,23.849,386,1018000,250026,341175
53423,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,2.0,NaN,...,0.0,0.0,1.0,0.0,21.810,21.810,392,1018001,250026,341176


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report, precision_score,recall_score,f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [3]:
CV_NAMES=['86032','91010','M 265','SNK-09293','PI 00-1110','CO VSI 08005']
df1=df[df.CV_NAME.isin (CV_NAMES)]
X1=df1[["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS"]].copy()
Y1=df1["DIVERSION_CENTER_NAME"].copy()
X1,Y1

(      CV_NAME  CLUSTER_NAME  PLANT_AREA CROP_TYPE_NAME  AGE_IN_DAYS
 0       91010  FACTORY EAST         4.0             R1          426
 1       91010  FACTORY EAST         5.0             R2          361
 2       91010  FACTORY EAST         5.0             R2          347
 3       86032  FACTORY EAST         6.0             PL          524
 4       M 265  FACTORY EAST         5.0             R1          387
 ...       ...           ...         ...            ...          ...
 53420   86032       LOKAPUR         2.0             R1           65
 53421   86032       LOKAPUR         1.0             PL          404
 53422   91010       LOKAPUR         1.0             PL          386
 53423   91010       LOKAPUR         1.0             PL          392
 53424   86032       LOKAPUR         1.0             PL          359
 
 [52775 rows x 5 columns],
 0                  GSM_HARVEST
 1                  GSM_HARVEST
 2                  GSM_HARVEST
 3        DUDHGANGA-KRISHNA SSK
 4             

In [4]:
for i in Y1.index:
    Y1[i]=(1 if (Y1[i]=="GSM_HARVEST") else 0)
Y1_new=Y1.astype('int')

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

features = ["CLUSTER_NAME","CROP_TYPE_NAME","CV_NAME"]
one = OneHotEncoder(sparse=False)
transformer = ColumnTransformer([("one",
                                  one,
                                  features)],
                                  remainder="passthrough")
X1_new = pd.DataFrame(transformer.fit_transform(X1))
X1_new,Y1_new

(        0    1    2    3    4    5    6    7    8    9   ...   21   22   23  \
 0      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 1      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 2      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 3      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
 4      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 52770  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  1.0  0.0  1.0   
 52771  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  1.0   
 52772  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 52773  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 52774  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  1.0   
 
         24   25   26   27   28   29  

In [5]:
X1train,X1test,Y1train,Y1test =train_test_split(X1_new,Y1_new,test_size=0.2)
def score(model):
    model.fit(X1train,Y1train)
    print(model.score(X1test,Y1test))

In [6]:
hgbc = HistGradientBoostingClassifier()
xgboost = XGBClassifier(use_label_encoder=False)
catboost = CatBoostClassifier()
lgbm = LGBMClassifier()
rf = RandomForestClassifier()
kn = KNeighborsClassifier()
lr = LogisticRegression(max_iter=4000)
rc = RidgeClassifier()
svc = SVC()
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()

In [23]:
score(rf)
score(lr)
score(kn)
score(rc)
score(svc)
score(bc)
score(gbc)
score(hgbc)
score(lgbm)
score(xgboost)
score(catboost)

0.8090004737091426
0.8097584083372809
0.81468498342018
0.7503552818569399
0.5708195168166745
0.8031264803410706
0.8360966366650876
0.8455708195168167
0.8439602084320227


C:\Users\91720\.conda\envs\env\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:21:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8414969208905732
Learning rate set to 0.050937
0:	learn: 0.6422751	total: 181ms	remaining: 3m 1s
1:	learn: 0.6040756	total: 192ms	remaining: 1m 35s
2:	learn: 0.5694444	total: 203ms	remaining: 1m 7s
3:	learn: 0.5409049	total: 214ms	remaining: 53.3s
4:	learn: 0.5156287	total: 225ms	remaining: 44.7s
5:	learn: 0.4967968	total: 236ms	remaining: 39.1s
6:	learn: 0.4836021	total: 247ms	remaining: 35.1s
7:	learn: 0.4716070	total: 258ms	remaining: 31.9s
8:	learn: 0.4652184	total: 268ms	remaining: 29.5s
9:	learn: 0.4549044	total: 279ms	remaining: 27.6s
10:	learn: 0.4476400	total: 289ms	remaining: 25.9s
11:	learn: 0.4410978	total: 299ms	remaining: 24.6s
12:	learn: 0.4353506	total: 310ms	rema

160:	learn: 0.3484694	total: 1.91s	remaining: 9.97s
161:	learn: 0.3484264	total: 1.93s	remaining: 9.96s
162:	learn: 0.3483232	total: 1.94s	remaining: 9.95s
163:	learn: 0.3482104	total: 1.95s	remaining: 9.93s
164:	learn: 0.3481351	total: 1.96s	remaining: 9.91s
165:	learn: 0.3479386	total: 1.97s	remaining: 9.9s
166:	learn: 0.3478600	total: 1.98s	remaining: 9.88s
167:	learn: 0.3477470	total: 1.99s	remaining: 9.87s
168:	learn: 0.3476180	total: 2s	remaining: 9.85s
169:	learn: 0.3475081	total: 2.01s	remaining: 9.83s
170:	learn: 0.3474290	total: 2.02s	remaining: 9.81s
171:	learn: 0.3473707	total: 2.03s	remaining: 9.79s
172:	learn: 0.3472935	total: 2.04s	remaining: 9.78s
173:	learn: 0.3472106	total: 2.06s	remaining: 9.76s
174:	learn: 0.3470723	total: 2.07s	remaining: 9.74s
175:	learn: 0.3470100	total: 2.08s	remaining: 9.72s
176:	learn: 0.3469313	total: 2.08s	remaining: 9.7s
177:	learn: 0.3468222	total: 2.1s	remaining: 9.68s
178:	learn: 0.3467163	total: 2.11s	remaining: 9.66s
179:	learn: 0.3465

324:	learn: 0.3356375	total: 3.65s	remaining: 7.58s
325:	learn: 0.3355969	total: 3.66s	remaining: 7.57s
326:	learn: 0.3355502	total: 3.67s	remaining: 7.56s
327:	learn: 0.3355049	total: 3.69s	remaining: 7.55s
328:	learn: 0.3354634	total: 3.69s	remaining: 7.54s
329:	learn: 0.3354210	total: 3.7s	remaining: 7.52s
330:	learn: 0.3353582	total: 3.71s	remaining: 7.51s
331:	learn: 0.3353160	total: 3.73s	remaining: 7.5s
332:	learn: 0.3352875	total: 3.74s	remaining: 7.49s
333:	learn: 0.3352251	total: 3.75s	remaining: 7.47s
334:	learn: 0.3351561	total: 3.76s	remaining: 7.46s
335:	learn: 0.3350995	total: 3.77s	remaining: 7.45s
336:	learn: 0.3350742	total: 3.78s	remaining: 7.43s
337:	learn: 0.3350278	total: 3.79s	remaining: 7.42s
338:	learn: 0.3349183	total: 3.8s	remaining: 7.41s
339:	learn: 0.3348221	total: 3.81s	remaining: 7.39s
340:	learn: 0.3347688	total: 3.82s	remaining: 7.38s
341:	learn: 0.3347370	total: 3.83s	remaining: 7.37s
342:	learn: 0.3346702	total: 3.84s	remaining: 7.36s
343:	learn: 0.3

483:	learn: 0.3284238	total: 5.34s	remaining: 5.69s
484:	learn: 0.3283918	total: 5.35s	remaining: 5.68s
485:	learn: 0.3283451	total: 5.36s	remaining: 5.67s
486:	learn: 0.3283109	total: 5.37s	remaining: 5.66s
487:	learn: 0.3282686	total: 5.38s	remaining: 5.64s
488:	learn: 0.3282288	total: 5.39s	remaining: 5.63s
489:	learn: 0.3281981	total: 5.4s	remaining: 5.62s
490:	learn: 0.3281616	total: 5.41s	remaining: 5.61s
491:	learn: 0.3281511	total: 5.42s	remaining: 5.6s
492:	learn: 0.3280939	total: 5.43s	remaining: 5.58s
493:	learn: 0.3280634	total: 5.44s	remaining: 5.57s
494:	learn: 0.3280167	total: 5.45s	remaining: 5.56s
495:	learn: 0.3279557	total: 5.46s	remaining: 5.55s
496:	learn: 0.3279185	total: 5.47s	remaining: 5.54s
497:	learn: 0.3278815	total: 5.48s	remaining: 5.53s
498:	learn: 0.3278627	total: 5.49s	remaining: 5.51s
499:	learn: 0.3278376	total: 5.5s	remaining: 5.5s
500:	learn: 0.3278263	total: 5.51s	remaining: 5.49s
501:	learn: 0.3277938	total: 5.52s	remaining: 5.48s
502:	learn: 0.32

650:	learn: 0.3228264	total: 7.08s	remaining: 3.8s
651:	learn: 0.3227835	total: 7.09s	remaining: 3.79s
652:	learn: 0.3227567	total: 7.1s	remaining: 3.77s
653:	learn: 0.3227231	total: 7.12s	remaining: 3.76s
654:	learn: 0.3226945	total: 7.13s	remaining: 3.75s
655:	learn: 0.3226620	total: 7.14s	remaining: 3.74s
656:	learn: 0.3226052	total: 7.15s	remaining: 3.73s
657:	learn: 0.3225777	total: 7.17s	remaining: 3.73s
658:	learn: 0.3225463	total: 7.18s	remaining: 3.71s
659:	learn: 0.3225060	total: 7.19s	remaining: 3.71s
660:	learn: 0.3224576	total: 7.21s	remaining: 3.7s
661:	learn: 0.3224138	total: 7.22s	remaining: 3.69s
662:	learn: 0.3223814	total: 7.25s	remaining: 3.68s
663:	learn: 0.3223575	total: 7.26s	remaining: 3.68s
664:	learn: 0.3223479	total: 7.28s	remaining: 3.67s
665:	learn: 0.3223034	total: 7.3s	remaining: 3.66s
666:	learn: 0.3222901	total: 7.32s	remaining: 3.65s
667:	learn: 0.3222462	total: 7.33s	remaining: 3.64s
668:	learn: 0.3222047	total: 7.35s	remaining: 3.63s
669:	learn: 0.32

814:	learn: 0.3180380	total: 9.48s	remaining: 2.15s
815:	learn: 0.3180221	total: 9.5s	remaining: 2.14s
816:	learn: 0.3180135	total: 9.52s	remaining: 2.13s
817:	learn: 0.3180059	total: 9.53s	remaining: 2.12s
818:	learn: 0.3179909	total: 9.54s	remaining: 2.11s
819:	learn: 0.3179770	total: 9.55s	remaining: 2.1s
820:	learn: 0.3179664	total: 9.57s	remaining: 2.09s
821:	learn: 0.3179488	total: 9.58s	remaining: 2.08s
822:	learn: 0.3179352	total: 9.6s	remaining: 2.06s
823:	learn: 0.3179123	total: 9.61s	remaining: 2.05s
824:	learn: 0.3178857	total: 9.62s	remaining: 2.04s
825:	learn: 0.3178539	total: 9.64s	remaining: 2.03s
826:	learn: 0.3178300	total: 9.65s	remaining: 2.02s
827:	learn: 0.3178095	total: 9.67s	remaining: 2.01s
828:	learn: 0.3177941	total: 9.69s	remaining: 2s
829:	learn: 0.3177759	total: 9.71s	remaining: 1.99s
830:	learn: 0.3177518	total: 9.72s	remaining: 1.98s
831:	learn: 0.3177373	total: 9.74s	remaining: 1.97s
832:	learn: 0.3177012	total: 9.76s	remaining: 1.96s
833:	learn: 0.3176

982:	learn: 0.3140805	total: 12.1s	remaining: 209ms
983:	learn: 0.3140586	total: 12.1s	remaining: 197ms
984:	learn: 0.3140438	total: 12.1s	remaining: 185ms
985:	learn: 0.3140273	total: 12.1s	remaining: 172ms
986:	learn: 0.3140149	total: 12.2s	remaining: 160ms
987:	learn: 0.3139857	total: 12.2s	remaining: 148ms
988:	learn: 0.3139557	total: 12.2s	remaining: 136ms
989:	learn: 0.3139336	total: 12.2s	remaining: 123ms
990:	learn: 0.3139148	total: 12.2s	remaining: 111ms
991:	learn: 0.3139060	total: 12.2s	remaining: 98.6ms
992:	learn: 0.3138714	total: 12.2s	remaining: 86.3ms
993:	learn: 0.3138565	total: 12.3s	remaining: 74ms
994:	learn: 0.3138454	total: 12.3s	remaining: 61.7ms
995:	learn: 0.3138209	total: 12.3s	remaining: 49.3ms
996:	learn: 0.3137921	total: 12.3s	remaining: 37ms
997:	learn: 0.3137749	total: 12.3s	remaining: 24.7ms
998:	learn: 0.3137549	total: 12.3s	remaining: 12.3ms
999:	learn: 0.3137279	total: 12.3s	remaining: 0us
0.8450971103742302


In [25]:

score(xgboost)

[19:30:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8414969208905732


In [34]:
X2train = X1train.drop(30,axis=1)
X2test = X1test.drop(30,axis=1)
def score2(model):
    model.fit(X2train,Y1train)
    print(model.score(X2test,Y1test))
X2train,X2test

(        0    1    2    3    4    5    6    7    8    9   ...   20   21   22  \
 22823  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  0.0  0.0   
 22646  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  1.0  0.0   
 46479  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 22826  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  0.0  0.0   
 34224  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 20587  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  1.0  0.0  0.0   
 9062   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 25233  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
 21196  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  1.0  0.0   
 19641  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  1.0   
 
         23   24   25   26   27   28  

In [35]:
score2(rf)
score2(lr)
score2(kn)
score2(rc)
score2(svc)
score2(bc)
score2(gbc)
score2(hgbc)
score2(lgbm)
score2(xgboost)
score2(catboost)

0.7458076740881099
0.7505447655139744
0.7180483183325438
0.7504500236854571
0.7532922785409758
0.7452392231170062
0.7551871151113216
0.7562292752250118
0.7561345333964945
[20:11:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7505447655139744
Learning rate set to 0.050937
0:	learn: 0.6770522	total: 10.9ms	remaining: 10.8s
1:	learn: 0.6619127	total: 21.9ms	remaining: 10.9s
2:	learn: 0.6491294	total: 33.2ms	remaining: 11s
3:	learn: 0.6371700	total: 44.5ms	remaining: 11.1s
4:	learn: 0.6269615	total: 54.7ms	remaining: 10.9s
5:	learn: 0.6178562	total: 65.9ms	remaining: 10.9s
6:	learn: 0.6096955	total: 75.9ms	remaining: 10.8s
7:	learn: 0.6021416	total: 84.4ms	remaining: 10.5s
8:	learn: 0.5954880	total: 94.1ms	remaining: 10.4s
9:	learn: 0.5

161:	learn: 0.5100573	total: 1.69s	remaining: 8.76s
162:	learn: 0.5100170	total: 1.7s	remaining: 8.76s
163:	learn: 0.5099425	total: 1.72s	remaining: 8.76s
164:	learn: 0.5099045	total: 1.73s	remaining: 8.75s
165:	learn: 0.5098710	total: 1.74s	remaining: 8.74s
166:	learn: 0.5098244	total: 1.75s	remaining: 8.73s
167:	learn: 0.5097923	total: 1.76s	remaining: 8.72s
168:	learn: 0.5097247	total: 1.77s	remaining: 8.71s
169:	learn: 0.5096680	total: 1.78s	remaining: 8.71s
170:	learn: 0.5096140	total: 1.79s	remaining: 8.7s
171:	learn: 0.5095518	total: 1.81s	remaining: 8.7s
172:	learn: 0.5094918	total: 1.82s	remaining: 8.69s
173:	learn: 0.5094332	total: 1.83s	remaining: 8.69s
174:	learn: 0.5093824	total: 1.84s	remaining: 8.68s
175:	learn: 0.5093424	total: 1.85s	remaining: 8.66s
176:	learn: 0.5093125	total: 1.86s	remaining: 8.65s
177:	learn: 0.5092732	total: 1.87s	remaining: 8.64s
178:	learn: 0.5092177	total: 1.88s	remaining: 8.62s
179:	learn: 0.5091814	total: 1.89s	remaining: 8.62s
180:	learn: 0.5

328:	learn: 0.5039168	total: 3.43s	remaining: 6.99s
329:	learn: 0.5038824	total: 3.44s	remaining: 6.98s
330:	learn: 0.5038584	total: 3.45s	remaining: 6.97s
331:	learn: 0.5038337	total: 3.46s	remaining: 6.96s
332:	learn: 0.5038113	total: 3.47s	remaining: 6.95s
333:	learn: 0.5037940	total: 3.48s	remaining: 6.93s
334:	learn: 0.5037557	total: 3.49s	remaining: 6.92s
335:	learn: 0.5037354	total: 3.5s	remaining: 6.92s
336:	learn: 0.5037192	total: 3.51s	remaining: 6.91s
337:	learn: 0.5036886	total: 3.52s	remaining: 6.9s
338:	learn: 0.5036602	total: 3.53s	remaining: 6.89s
339:	learn: 0.5036350	total: 3.54s	remaining: 6.88s
340:	learn: 0.5036059	total: 3.55s	remaining: 6.87s
341:	learn: 0.5035750	total: 3.56s	remaining: 6.86s
342:	learn: 0.5035475	total: 3.57s	remaining: 6.84s
343:	learn: 0.5035197	total: 3.58s	remaining: 6.83s
344:	learn: 0.5034989	total: 3.59s	remaining: 6.82s
345:	learn: 0.5034691	total: 3.6s	remaining: 6.81s
346:	learn: 0.5034577	total: 3.62s	remaining: 6.8s
347:	learn: 0.50

498:	learn: 0.5003140	total: 5.16s	remaining: 5.18s
499:	learn: 0.5002933	total: 5.17s	remaining: 5.17s
500:	learn: 0.5002792	total: 5.18s	remaining: 5.16s
501:	learn: 0.5002648	total: 5.19s	remaining: 5.15s
502:	learn: 0.5002507	total: 5.2s	remaining: 5.14s
503:	learn: 0.5002415	total: 5.21s	remaining: 5.13s
504:	learn: 0.5002268	total: 5.22s	remaining: 5.12s
505:	learn: 0.5002115	total: 5.23s	remaining: 5.11s
506:	learn: 0.5002066	total: 5.24s	remaining: 5.1s
507:	learn: 0.5001973	total: 5.25s	remaining: 5.09s
508:	learn: 0.5001867	total: 5.26s	remaining: 5.08s
509:	learn: 0.5001647	total: 5.27s	remaining: 5.07s
510:	learn: 0.5001621	total: 5.28s	remaining: 5.05s
511:	learn: 0.5001570	total: 5.29s	remaining: 5.04s
512:	learn: 0.5001333	total: 5.3s	remaining: 5.03s
513:	learn: 0.5001168	total: 5.31s	remaining: 5.02s
514:	learn: 0.5000995	total: 5.32s	remaining: 5.01s
515:	learn: 0.5000813	total: 5.33s	remaining: 5s
516:	learn: 0.5000626	total: 5.34s	remaining: 4.99s
517:	learn: 0.5000

668:	learn: 0.4977325	total: 6.9s	remaining: 3.41s
669:	learn: 0.4977247	total: 6.91s	remaining: 3.4s
670:	learn: 0.4977088	total: 6.92s	remaining: 3.39s
671:	learn: 0.4976895	total: 6.93s	remaining: 3.38s
672:	learn: 0.4976784	total: 6.94s	remaining: 3.37s
673:	learn: 0.4976647	total: 6.95s	remaining: 3.36s
674:	learn: 0.4976383	total: 6.96s	remaining: 3.35s
675:	learn: 0.4976262	total: 6.97s	remaining: 3.34s
676:	learn: 0.4976193	total: 6.98s	remaining: 3.33s
677:	learn: 0.4975999	total: 6.99s	remaining: 3.32s
678:	learn: 0.4975851	total: 7s	remaining: 3.31s
679:	learn: 0.4975792	total: 7.01s	remaining: 3.3s
680:	learn: 0.4975646	total: 7.02s	remaining: 3.29s
681:	learn: 0.4975493	total: 7.03s	remaining: 3.28s
682:	learn: 0.4975330	total: 7.04s	remaining: 3.27s
683:	learn: 0.4975287	total: 7.05s	remaining: 3.26s
684:	learn: 0.4975117	total: 7.06s	remaining: 3.25s
685:	learn: 0.4975024	total: 7.07s	remaining: 3.24s
686:	learn: 0.4974940	total: 7.08s	remaining: 3.23s
687:	learn: 0.4974

839:	learn: 0.4957170	total: 8.65s	remaining: 1.65s
840:	learn: 0.4957029	total: 8.66s	remaining: 1.64s
841:	learn: 0.4956950	total: 8.67s	remaining: 1.63s
842:	learn: 0.4956755	total: 8.68s	remaining: 1.62s
843:	learn: 0.4956621	total: 8.69s	remaining: 1.61s
844:	learn: 0.4956504	total: 8.7s	remaining: 1.59s
845:	learn: 0.4956422	total: 8.71s	remaining: 1.58s
846:	learn: 0.4956364	total: 8.72s	remaining: 1.57s
847:	learn: 0.4956282	total: 8.73s	remaining: 1.56s
848:	learn: 0.4956138	total: 8.74s	remaining: 1.55s
849:	learn: 0.4956064	total: 8.75s	remaining: 1.54s
850:	learn: 0.4956010	total: 8.76s	remaining: 1.53s
851:	learn: 0.4955897	total: 8.77s	remaining: 1.52s
852:	learn: 0.4955782	total: 8.78s	remaining: 1.51s
853:	learn: 0.4955628	total: 8.79s	remaining: 1.5s
854:	learn: 0.4955478	total: 8.8s	remaining: 1.49s
855:	learn: 0.4955320	total: 8.81s	remaining: 1.48s
856:	learn: 0.4955202	total: 8.82s	remaining: 1.47s
857:	learn: 0.4955124	total: 8.83s	remaining: 1.46s
858:	learn: 0.4

0.7555660824253908


In [44]:
def gs(model,params):
    gs = GridSearchCV(model,param_grid=params,cv=5,verbose=2)
    gs.fit(X1train,Y1train)
    Y1preds=gs.predict(X1test)
    print(pd.DataFrame(classification_report(Y1test,Y1preds, output_dict=True)).transpose())
    print(confusion_matrix(Y1test,Y1preds))
    print(gs.score(X1test,Y1test), gs.best_params_,gs.scorer_)

In [ ]:
params={}
gs(HistGradientBoostingClassifier(),params)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.0s


In [ ]:
0.8346755092373283 {}